In [1]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Single sheet attendance data processing script
import pandas as pd
import os
# load data from excel
file = 'exported1.xlsx'
df = pd.read_excel(f'data/single/{file}', engine='openpyxl')

# remove first row
df = df.iloc[1:]

# create a new data frame with columns: name, department, period, date, clock-in time, clock-out time
formatted_df = pd.DataFrame(columns=["name", "department", "period", "date", "clock-in", "clock-out"])

# get period from col 3 and row 1, remove '\t( babyloop )'
period = df.iloc[0, 2].replace('\t( babyloop )', '').strip()

# remove first row after getting period
df = df.iloc[1:]

# function to format the attendance DataFrame
def format_attende_df(df_in):
    """
    Format the attendance DataFrame by extracting name, department, dates, clock-in and clock-out times.
    """
    # Extract name and department from row 1 (index 3)
    name = df_in.iloc[1, 10]
    department = df_in.iloc[1, 14]


    # Loop through columns in row 0 (index 2) to extract all non-null dates
    dates = []
    prev_val = 0
    # add month and year to 
    # the date from period variable(e.g., period = '2025/06/26 ~ 07/04' then month = '06', year = '2025', date = '26' first var record should be '26' from period extraction)
    year, month, date = period.split('/')[0], period.split('/')[1], period.split('/')[2].split(' ')[1]
    for col in sample_df.columns:
        val = sample_df.iloc[0][col]
        if pd.notnull(val) and str(val).strip():
            if prev_val > val:
                month = str(int(month) + 1).zfill(2)  # Increment month if current value is less than previous
            prev_val = val
            # remove leading zeros and convert to string
            val = str(int(val)).zfill(2)  # Ensure date is two digits
            # Construct the date string in the format 'YYYY/MM/DD'
            date = f"{year}/{month}/{str(val).strip()}"
            # Append the date to the list
            dates.append(date)
            # dates.append(str(val).strip())

    # Extract clock-in and clock-out for each date column from row 2 (index 4)
    attende_df = pd.DataFrame(columns=["date", "clock-in", "clock-out"])
    for i, col in enumerate(df_in.columns[:len(dates)]):
        times_str = df_in.iloc[2][col]
        times = [t.strip() for t in str(times_str).split('\n') if t.strip()]
        if len(times) == 4:
            # If there are 4 times, split into two rows
            attende_df = pd.concat([
                attende_df,
                pd.DataFrame([
                    {"date": dates[i], "clock-in": times[0], "clock-out": times[1]},
                    {"date": dates[i], "clock-in": times[2], "clock-out": times[3]}
                ])
            ], ignore_index=True)
        elif len(times) == 3:
            if times[0][:2] == times[1][:2]:
                attende_df = pd.concat([
                    attende_df,
                    pd.DataFrame([{"date": dates[i], "clock-in": times[0], "clock-out": times[2]}])
                ], ignore_index=True)
            else:
                attende_df = pd.concat([
                    attende_df,
                    pd.DataFrame([
                        {"date": dates[i], "clock-in": times[0], "clock-out": times[1]},
                        {"date": dates[i], "clock-in": times[1], "clock-out": times[2]}
                    ])
                ], ignore_index=True)
        else:
            clock_in = times[0] if len(times) > 0 else ''
            clock_out = times[1] if len(times) > 1 else ''
            attende_df = pd.concat([
                attende_df,
                pd.DataFrame([{"date": dates[i], "clock-in": clock_in, "clock-out": clock_out}])
            ], ignore_index=True)

    def clean_time(val):
        if val is None or str(val).lower() == 'nan':
            return ''
        return val

    df_out = pd.DataFrame([{
        "name": name,
        "department": department,
        "period": period,
        "date": row["date"],
        "dayofweek": pd.to_datetime(row["date"]).day_name() if row["date"] else '',
        "clock-in": clean_time(row["clock-in"]),
        "clock-out": clean_time(row["clock-out"])
    } for _, row in attende_df.iterrows()])

    return df_out

# process df disect it to 3 rows then process it using format_attende_df, append result formatted_df
exported_df = df.copy()
# process exported_df to disect it to 3 rows
# then process it using format_attende_df, append result formatted_df

print("Processing exported_df to disect it to 3 rows and format it...")
formatted_df = pd.DataFrame(columns=["name", "department", "period", "date", "clock-in", "clock-out"])
for i in range(0, len(exported_df), 3):
    sample_df = exported_df.iloc[i:i+3]
    if sample_df.shape[0] < 3:
        continue  # Skip if not enough rows
    formatted_sample_df = format_attende_df(sample_df)
    formatted_df = pd.concat([formatted_df, formatted_sample_df], ignore_index=True)

# enrich data
# add column is_late if clock-in time is after 8:00 AM and clock-out time is not empty
# add column late_duration which is the difference between clock-in time and 8:00 AM
formatted_df['is_late'] = formatted_df.apply(
    lambda row: pd.to_datetime(row['clock-in'], errors='coerce') > pd.to_datetime('08:00:00', errors='coerce') 
    and row['clock-out'] != '', axis=1)
formatted_df['late_duration'] = formatted_df.apply(
    lambda row: (pd.to_datetime(row['clock-in'], errors='coerce') - pd.to_datetime('08:00:00', errors='coerce')).total_seconds() / 60 
    if row['is_late'] else 0, axis=1)
# Overtime calculation
# add column overtime if clock-out time is after 4:15 PM and clock-in time is not empty
formatted_df['overtime'] = formatted_df.apply(
    lambda row: pd.to_datetime(row['clock-out'], errors='coerce') > pd.to_datetime('16:15:00', errors='coerce') 
    and row['clock-in'] != '', axis=1)
formatted_df['overtime_duration'] = formatted_df.apply(
    lambda row: (pd.to_datetime(row['clock-out'], errors='coerce') - pd.to_datetime('16:15:00', errors='coerce')).total_seconds() / 60 
    if row['overtime'] else 0, axis=1)
# add column is_absent if clock-in time is empty and clock-out time is empty
formatted_df['is_absent'] = formatted_df.apply(
    lambda row: row['clock-in'] == '' and row['clock-out'] == '', axis=1)

# Output to directory
# created directory if not exists
if not os.path.exists('formatted/single'):
    os.makedirs('formatted/single')
base_filename = file.split('.')[0]  # Get the base filename without extension
formatted_df.to_csv(f'formatted/single/{base_filename}.csv', index=False)


Processing exported_df to disect it to 3 rows and format it...


In [1]:
# Multiple spreadsheet attendance data processing script
import os
import pandas as pd

# LOAD DATA
# load data from excel
file = 'Absensi Produksi 2025 07.xlsx'

# get sheet names
sheet_names = pd.ExcelFile(f'data/compiled/{file}', engine='openpyxl').sheet_names
print(sheet_names)

dfs = {}
for sheet in sheet_names:
    # if sheet starts with 'Log', take it and put it in dfs
    if sheet.startswith('Log'):
        print(f"Loading sheet: {sheet}")
        df = pd.read_excel(f'data/compiled/{file}', sheet_name=sheet, engine='openpyxl')
        dfs[sheet] = df

def prepare_df_in(df):
    """
    Prepare the DataFrame by removing the first row and extracting the period.
    """
    # remove first row
    df = df.iloc[1:]

    # create a new data frame with columns: name, department, period, date, clock-in time, clock-out time
    formatted_df = pd.DataFrame(columns=["name", "department", "period", "date", "clock-in", "clock-out"])

    # get period from col 3 and row 1, remove '\t( babyloop )'
    period = df.iloc[0, 2].replace('\t( babyloop )', '').strip()

    # remove first row after getting period
    df = df.iloc[1:]

    return df, formatted_df, period

dfs_in = {}
for sheet, df in dfs.items():
    print(f"Processing sheet: {sheet}")
    df_in, formatted_df, period = prepare_df_in(df)
    dfs_in[sheet] = (df_in, formatted_df, period)


# function to format the attendance DataFrame
def format_attende_df(df_in):
    """
    Format the attendance DataFrame by extracting name, department, dates, clock-in and clock-out times.
    """
    # Extract name and department from row 1 (index 3)
    name = df_in.iloc[1, 10]
    department = df_in.iloc[1, 14]


    # Loop through columns in row 0 (index 2) to extract all non-null dates
    dates = []
    prev_val = 0
    # add month and year to 
    # the date from period variable(e.g., period = '2025/06/26 ~ 07/04' then month = '06', year = '2025', date = '26' first var record should be '26' from period extraction)
    year, month, date = period.split('/')[0], period.split('/')[1], period.split('/')[2].split(' ')[1]
    for col in df_in.columns:
        val = df_in.iloc[0][col]
        if pd.notnull(val) and str(val).strip():
            if prev_val > val:
                month = str(int(month) + 1).zfill(2)  # Increment month if current value is less than previous
            prev_val = val
            # remove leading zeros and convert to string
            val = str(int(val)).zfill(2)  # Ensure date is two digits
            # Construct the date string in the format 'YYYY/MM/DD'
            date = f"{year}/{month}/{str(val).strip()}"
            # Append the date to the list
            dates.append(date)
            # dates.append(str(val).strip())

    # Extract clock-in and clock-out for each date column from row 2 (index 4)
    attende_df = pd.DataFrame(columns=["date", "clock-in", "clock-out"])
    for i, col in enumerate(df_in.columns[:len(dates)]):
        times_str = df_in.iloc[2][col]
        times = [t.strip() for t in str(times_str).split('\n') if t.strip()]
        if len(times) == 4:
            # If there are 4 times, split into two rows
            attende_df = pd.concat([
                attende_df,
                pd.DataFrame([
                    {"date": dates[i], "clock-in": times[0], "clock-out": times[1]},
                    {"date": dates[i], "clock-in": times[2], "clock-out": times[3]}
                ])
            ], ignore_index=True)
        elif len(times) == 3:
            if times[0][:2] == times[1][:2]:
                attende_df = pd.concat([
                    attende_df,
                    pd.DataFrame([{"date": dates[i], "clock-in": times[0], "clock-out": times[2]}])
                ], ignore_index=True)
            else:
                attende_df = pd.concat([
                    attende_df,
                    pd.DataFrame([
                        {"date": dates[i], "clock-in": times[0], "clock-out": times[1]},
                        {"date": dates[i], "clock-in": times[1], "clock-out": times[2]}
                    ])
                ], ignore_index=True)
        else:
            clock_in = times[0] if len(times) > 0 else ''
            clock_out = times[1] if len(times) > 1 else ''
            attende_df = pd.concat([
                attende_df,
                pd.DataFrame([{"date": dates[i], "clock-in": clock_in, "clock-out": clock_out}])
            ], ignore_index=True)

    def clean_time(val):
        if val is None or str(val).lower() == 'nan':
            return ''
        return val

    df_out = pd.DataFrame([{
        "name": name,
        "department": department,
        "period": period,
        "date": row["date"],
        "dayofweek": pd.to_datetime(row["date"]).day_name() if row["date"] else '',
        "clock-in": clean_time(row["clock-in"]),
        "clock-out": clean_time(row["clock-out"])
    } for _, row in attende_df.iterrows()])

    return df_out

def process_sheet(df, formatted_df, period):
    """
    Process the DataFrame by extracting attendance data and formatting it.
    """
    # Loop through the DataFrame in chunks of 3 rows
    for i in range(0, len(df), 3):
        sample_df = df.iloc[i:i+3]
        if sample_df.shape[0] < 3:
            continue  # Skip if not enough rows
        formatted_sample_df = format_attende_df(sample_df)
        formatted_df = pd.concat([formatted_df, formatted_sample_df], ignore_index=True)
    
    return formatted_df

formatted_dfs = {}
for sheet, (df_in, formatted_df, period) in dfs_in.items():
    print(f"Processing sheet: {sheet}")
    formatted_df = process_sheet(df_in, formatted_df, period)
    formatted_dfs[sheet] = formatted_df

def enrich_data(formatted_df):
    """
    Enrich the attendance DataFrame with additional columns for late, overtime, and absence.
    """
    # add column is_late if clock-in time is after 8:00 AM and clock-out time is not empty
    # add column late_duration which is the difference between clock-in time and 8:00 AM
    formatted_df['is_late'] = formatted_df.apply(
        lambda row: pd.to_datetime(row['clock-in'], errors='coerce') > pd.to_datetime('08:00:00', errors='coerce') 
        and row['clock-out'] != '', axis=1)
    formatted_df['late_duration'] = formatted_df.apply(
        lambda row: (pd.to_datetime(row['clock-in'], errors='coerce') - pd.to_datetime('08:00:00', errors='coerce')).total_seconds() / 60 
        if row['is_late'] else 0, axis=1)
    
    # Overtime calculation
    # For Monday to Friday, overtime if clock-out is after 16:15; for Saturday, after 14:00
    def is_overtime(row):
        if row['clock-in'] == '' or row['clock-out'] == '':
            return False
        if row['dayofweek'] == 'Saturday':
            return pd.to_datetime(row['clock-out'], errors='coerce') > pd.to_datetime('14:00:00', errors='coerce')
        elif row['dayofweek'] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            return pd.to_datetime(row['clock-out'], errors='coerce') > pd.to_datetime('16:15:00', errors='coerce')
        else:
            return False

    def overtime_duration(row):
        if not row['overtime']:
            return 0
        if row['dayofweek'] == 'Saturday':
            return (pd.to_datetime(row['clock-out'], errors='coerce') - pd.to_datetime('14:00:00', errors='coerce')).total_seconds() / 60
        elif row['dayofweek'] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            return (pd.to_datetime(row['clock-out'], errors='coerce') - pd.to_datetime('16:15:00', errors='coerce')).total_seconds() / 60
        else:
            return 0

    formatted_df['overtime'] = formatted_df.apply(is_overtime, axis=1)
    formatted_df['overtime_duration'] = formatted_df.apply(overtime_duration, axis=1)
    
    # add column is_absent if clock-in time is empty and clock-out time is empty
    formatted_df['is_absent'] = formatted_df.apply(
        lambda row: row['clock-in'] == '' and row['clock-out'] == '', axis=1)

    return formatted_df

formatted_dfs_enriched = {}
for sheet, formatted_df in formatted_dfs.items():
    print(f"Enriching data for sheet: {sheet}")
    enriched_df = enrich_data(formatted_df)
    formatted_dfs_enriched[sheet] = enriched_df

compiled_df = pd.concat(formatted_dfs_enriched.values(), ignore_index=True)

if not os.path.exists('formatted/compiled'):
    os.makedirs('formatted/compiled')
# Save the formatted DataFrame to an Excel file
output_file = 'compiled_attendance.xlsx'
compiled_df.to_excel(f'formatted/compiled/{output_file}', index=False, engine='openpyxl')

['4 Juli', 'Log 4', 'Sheet1', '11 Juli', 'Log 11', 'Sheet2', '18 Juli', 'Log 18', '25 Juli', 'Kurang', 'Log 25', 'Sheet5', 'Sheet3', 'Rekap Lembur']
Loading sheet: Log 4
Loading sheet: Log 11
Loading sheet: Log 18
Loading sheet: Log 25
Processing sheet: Log 4
Processing sheet: Log 11
Processing sheet: Log 18
Processing sheet: Log 25
Processing sheet: Log 4
Processing sheet: Log 11
Processing sheet: Log 18
Processing sheet: Log 25
Enriching data for sheet: Log 4
Enriching data for sheet: Log 11
Enriching data for sheet: Log 18
Enriching data for sheet: Log 25
